# Applications of Vectorization

# Spam Filter - Count Vectorization Method

In [1]:
import pandas as pd
import numpy as np

## Import the Data

Import the data and take a look at it.

In [2]:
url = "https://raw.githubusercontent.com/ryanleeallred/datasets/master/spam.csv"

df = pd.read_csv(url, encoding="ISO-8859-1")
print(df.shape)
df.head()

(5572, 5)


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## Tidy up initial DataFrame

- Change Pandas display options so that we can see more of the text
- Drop the unnamed columns, I'm not sure why they're in there, but we don't need them.
- Rename the v1 and v2 columns.

In [3]:
pd.set_option('display.max_colwidth', 200)
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
df = df.rename(columns={"v1":"label", "v2":"text"})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


You'll notice right of the bat that this text isn't as coherent as the job listings. We'll proceed like normal though. 

What is the ratio of Spam to Ham messages?

In [4]:
df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

## Categorical encoding on labels.

In [5]:
df['label_num'] = df.label.map({'ham': 0, 'spam': 1})

df.head()

,label,text,label_num
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives around here though",0


## Model Validation - Train Test Split (quick and dirty)
Since we're going to do some modeling we're going to need some model validation. For simplicity lets just do a quick train_test_split for today. You can try out Cross Validation on your assignment today, I just want to get to a quick baseline. 

In [6]:
from sklearn.model_selection import train_test_split

X = df.text
y = df.label_num

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Look at sizes of our train and test datasets

In [7]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4457,)
(1115,)
(4457,)
(1115,)


## Count Vectorizer

Today we're just going to let Scikit-Learn do our text cleaning and preprocessing for us.

Lets run our vectorizer on our text messages and take a peek at the tokenization of the vocabulary

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)

{'boat': 1371, 'moms': 4416, 'check': 1706, 'yo': 7415, 'half': 3210, 'naked': 4520, 'bank': 1187, 'granite': 3140, 'issues': 3631, 'strong': 6296, 'buy': 1533, 'explosive': 2661, 'pick': 5005, 'members': 4292, '300': 419, 'nasdaq': 4533, 'symbol': 6440, 'cdgt': 1649, '00': 0, 'giving': 3061, 'second': 5770, 'chance': 1674, 'rahul': 5351, 'dengra': 2190, 'played': 5048, 'smash': 6027, 'bros': 1476, 'lt': 4109, 'gt': 3174, 'religiously': 5484, 'private': 5225, '2003': 345, 'account': 781, 'statement': 6232, '07973788240': 45, 'shows': 5921, '800': 630, 'redeemed': 5453, 'points': 5096, '08715203649': 122, 'identifier': 3483, 'code': 1831, '40533': 476, 'expires': 2656, '31': 430, '10': 255, '04': 15, 'says': 5728, 'answer': 954, 'texts': 6566, 'confirm': 1912, 'deny': 2196, '88066': 695, 'lost': 4073, '3pound': 463, 'help': 3301, 'okey': 4742, 'dokey': 2346, 'll': 4022, 'bit': 1323, 'just': 3752, 'sorting': 6106, 'stuff': 6311, 'come': 1858, 'people': 4953, 'wah': 7083, 'lucky': 4115, '

Now we'll complete the vectorization by running .transform() and then save the results to a dataframe for viewing.
You don't need to save it to a dataframe, you can use most ML models with just the 2D array output.

That's a lot of columns.

In [10]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(4457, 7472)


,00,000,000pes,008704050406,0089,0121,01223585236,0125698789,02,0207,...,ìï,û_,û_thanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We also need to vectorize our X_test data, but we need to use the same vocabulary as the training dataset, so we'll just call .transform() on X_test to get our vectorized X_test_df

In [12]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(1115, 7472)


,00,000,000pes,008704050406,0089,0121,01223585236,0125698789,02,0207,...,ìï,û_,û_thanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Lets run some classification models and see what kind of accuracy we can get!

## Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

/anaconda3/envs/sandbox/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Now we'll evaluate both our training and testing accuracy. 

In [14]:
from sklearn.metrics import accuracy_score

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.9964101413506843
Test Accuracy: 0.9775784753363229


## Multinomial Naive Bayes

In [15]:
from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.9946152120260264
Test Accuracy: 0.9838565022421525


## Random Forest Classifier

In [16]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

/anaconda3/envs/sandbox/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train Accuracy: 0.9968588736818488
Test Accuracy: 0.9695067264573991


# Spam Filter - TF-IDF Vectorization Method

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)

{'boat': 1371, 'moms': 4416, 'check': 1706, 'yo': 7415, 'half': 3210, 'naked': 4520, 'bank': 1187, 'granite': 3140, 'issues': 3631, 'strong': 6296, 'buy': 1533, 'explosive': 2661, 'pick': 5005, 'members': 4292, '300': 419, 'nasdaq': 4533, 'symbol': 6440, 'cdgt': 1649, '00': 0, 'giving': 3061, 'second': 5770, 'chance': 1674, 'rahul': 5351, 'dengra': 2190, 'played': 5048, 'smash': 6027, 'bros': 1476, 'lt': 4109, 'gt': 3174, 'religiously': 5484, 'private': 5225, '2003': 345, 'account': 781, 'statement': 6232, '07973788240': 45, 'shows': 5921, '800': 630, 'redeemed': 5453, 'points': 5096, '08715203649': 122, 'identifier': 3483, 'code': 1831, '40533': 476, 'expires': 2656, '31': 430, '10': 255, '04': 15, 'says': 5728, 'answer': 954, 'texts': 6566, 'confirm': 1912, 'deny': 2196, '88066': 695, 'lost': 4073, '3pound': 463, 'help': 3301, 'okey': 4742, 'dokey': 2346, 'll': 4022, 'bit': 1323, 'just': 3752, 'sorting': 6106, 'stuff': 6311, 'come': 1858, 'people': 4953, 'wah': 7083, 'lucky': 4115, '

## Vectorize training data

In [18]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(4457, 7472)


,00,000,000pes,008704050406,0089,0121,01223585236,0125698789,02,0207,...,ìï,û_,û_thanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.265494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Vectorize testing data

In [19]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(1115, 7472)


,00,000,000pes,008704050406,0089,0121,01223585236,0125698789,02,0207,...,ìï,û_,û_thanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Logistic Regression

In [20]:
LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

/anaconda3/envs/sandbox/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train Accuracy: 0.9694862014808167
Test Accuracy: 0.9524663677130045


## Multinomial Naive Bayes

In [21]:
from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.9838456360780794
Test Accuracy: 0.9668161434977578


## Random Forest Classifier

In [22]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

/anaconda3/envs/sandbox/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train Accuracy: 0.9975319721785955
Test Accuracy: 0.9739910313901345


# Sentiment Analysis

## What is Sentiment Analysis?

The objective of sentiment analysis is to take a phrase and based on the text of the phrase determine if its sentiment is: Postive, Neutral, or Negative. 

Suppose that you wanted to use NLP to classify reviews for your company's products as either positive, neutral, or negative. Maybe you don't trust the star ratings left by the users and you want an additional measure of sentiment from each review - maybe you would use this as a feature generation technique for additional modeling, or to identify disgruntled customers and reach out to them to improve your customer service, etc. Sentiment Analysis has also been used heavily in stock market price estimation by trying to track the sentiment of the tweets of individuals after breaking news comes out about a company.

Does every word in each review contribute to its overall sentiment? Not really. Stop words for example don't really tell us much about the overall sentiment of the text, so just like we did before, we will discard them. 

## NLTK Movie Review Sentiment Analysis

In [23]:
# !pip install -U nltk

import nltk
# nltk.download('movie_reviews')
# nltk.download('stopwords')
from nltk.corpus import movie_reviews
import random

## Check that we have movie reviews

In [24]:
# How many total reviews are there?
print("Total reviews:", len(movie_reviews.fileids()))

# Total positive reviews
print("Positive reviews:", len(movie_reviews.fileids('pos'))) 
 
# Total negative reviews
print("Negative reviews:", len(movie_reviews.fileids('neg')))

Total reviews: 2000
Positive reviews: 1000
Negative reviews: 1000


## Get Reviews and randomize

In [25]:
reviews = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]

random.shuffle(reviews)

## Understand the format of the data

In [26]:
# Print Review Text:
print(reviews[0][0])

# Print Review Sentiment:
print(reviews[0][1])

# Print Review Text:
print(reviews[1][0])

# Print Review Sentiment:
print(reviews[1][1])

['the', 'word', "'", 'rest', "'", 'in', 'the', 'title', 'should', 'be', 'stressed', 'since', 'this', 'is', 'a', 'movie', 'that', 'lacks', 'any', 'action', 'scenes', 'at', 'all', '.', 'what', 'does', 'takes', 'place', 'is', 'more', 'based', 'on', 'character', 'development', '.', 'the', 'film', 'starts', 'right', 'in', 'the', 'middle', 'of', 'the', 'lives', 'of', 'four', 'people', ':', 'nick', ',', 'beth', ',', 'carol', 'and', 'sid', '.', 'what', 'they', 'have', 'done', 'earlier', 'is', 'for', 'the', 'most', 'part', 'only', 'hinted', 'at', '.', 'nick', 'and', 'beth', 'have', 'a', 'relationship', 'and', 'have', 'decided', 'to', 'move', 'from', 'enfield', ',', 'arizona', 'to', 'butte', ',', 'montana', '.', '(', 'and', 'i', 'who', 'thought', 'that', 'butte', 'was', 'pronounced', 'slightly', 'different', '.', '.', '.', ')', 'carol', 'is', 'nick', "'", 's', 'ex', '-', 'girlfriend', 'and', 'beth', "'", 's', 'best', 'friend', ',', 'sid', 'is', 'a', 'joint', '-', 'smoking', 'painter', 'that', 'c

## Add reviews to a dataframe for kicks

In [27]:
documents = []
sentiments = []

for review in reviews:
  
  # Add sentiment to list
  if review[1] == "pos":
    sentiments.append(1)
  else:
    sentiments.append(0)
  
  # Add text to list
  review_text = " ".join(review[0])
  documents.append(review_text)
  
df = pd.DataFrame({"text": documents, "sentiment": sentiments})
df.head()

,text,sentiment
0,the word ' rest ' in the title should be stressed since this is a movie that lacks any action scenes at all . what does takes place is more based on character development . the film starts right i...,1
1,"truman ( "" true - man "" ) burbank is the perfect name for jim carrey ' s character in this film . president truman was an unassuming man who became known worldwide , in spite of ( or was it becaus...",1
2,"i am starting to write this review before going to see strawberry and chocolate , cuba ' s oscar nominee for best foreign film , directed by tomas gutierrez alea ( who also directed the sometimes ...",1
3,"perhaps it ' s time for me to say a little about my reading habits . i really do like to read , and i ' ve enjoyed many books in my lifetime . my problem is that i ' m a slow reader , and not very...",1
4,"not since attending an ingmar bergman retrospective a few years ago have i seen a film as uncompromising in its portrayal of emotional truth as secrets & lies . like bergman , director mike leigh ...",1


## Train Test Split

In [28]:
X = df.text
y = df.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sentiment Analysis - CountVectorizer

## Generate vocabulary from train dataset

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)

{'ingredients': 16419, 'possessed': 24407, 'plastic': 24020, 'dolls': 9500, 'love': 19098, 'having': 14635, 'sex': 28551, 'starring': 30465, 'jennifer': 17212, 'tilly': 32442, 'voice': 34732, 'brad': 4129, 'dourif': 9649, 'katherine': 17602, 'heigl': 14805, 'nick': 21675, 'stabile': 30346, 'john': 17337, 'ritter': 27103, 'synopsis': 31623, 'fourth': 12668, 'film': 12043, 'chucky': 5809, 'series': 28483, 'debuted': 8150, 'late': 18254, '1980s': 179, 'basically': 2955, 'doll': 9492, 'walk': 34871, 'talk': 31729, 'spirit': 30122, 'slain': 29328, 'murderer': 21202, 'bride': 4301, 'longtime': 18999, 'girlfriend': 13452, 'tiffany': 32423, 'dies': 8883, 'inhabits': 16427, 'female': 11882, 'voodoo': 34771, 'married': 19716, 'embark': 10447, 'quest': 25450, 'reach': 25807, 'cemetery': 5342, 'new': 21610, 'jersey': 17236, 'mystical': 21296, 'gem': 13241, 'enable': 10566, 'humans': 15538, 'stow': 30751, 'away': 2570, 'vehicle': 34374, 'driven': 9825, 'newly': 21626, 'eloped': 10420, 'couple': 730

## Generate Vectorizations

In [30]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(1600, 36094)


,00,000,0009f,007,03,04,05,05425,10,100,...,zuehlke,zuko,zukovsky,zulu,zurg,zus,zweibel,zwick,zwigoff,zycie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(400, 36094)


,00,000,0009f,007,03,04,05,05425,10,100,...,zuehlke,zuko,zukovsky,zulu,zurg,zus,zweibel,zwick,zwigoff,zycie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Logistic Regression

In [32]:
LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

/anaconda3/envs/sandbox/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train Accuracy: 1.0
Test Accuracy: 0.835


## Multinomial Naive Bayes

In [33]:
MNB = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.980625
Test Accuracy: 0.7775


## Random Forest Classifier

In [34]:
RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

/anaconda3/envs/sandbox/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train Accuracy: 0.9925
Test Accuracy: 0.645


# Sentiment Analysis - tfidfVectorizer

## Vocabulary

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)

{'ingredients': 16419, 'possessed': 24407, 'plastic': 24020, 'dolls': 9500, 'love': 19098, 'having': 14635, 'sex': 28551, 'starring': 30465, 'jennifer': 17212, 'tilly': 32442, 'voice': 34732, 'brad': 4129, 'dourif': 9649, 'katherine': 17602, 'heigl': 14805, 'nick': 21675, 'stabile': 30346, 'john': 17337, 'ritter': 27103, 'synopsis': 31623, 'fourth': 12668, 'film': 12043, 'chucky': 5809, 'series': 28483, 'debuted': 8150, 'late': 18254, '1980s': 179, 'basically': 2955, 'doll': 9492, 'walk': 34871, 'talk': 31729, 'spirit': 30122, 'slain': 29328, 'murderer': 21202, 'bride': 4301, 'longtime': 18999, 'girlfriend': 13452, 'tiffany': 32423, 'dies': 8883, 'inhabits': 16427, 'female': 11882, 'voodoo': 34771, 'married': 19716, 'embark': 10447, 'quest': 25450, 'reach': 25807, 'cemetery': 5342, 'new': 21610, 'jersey': 17236, 'mystical': 21296, 'gem': 13241, 'enable': 10566, 'humans': 15538, 'stow': 30751, 'away': 2570, 'vehicle': 34374, 'driven': 9825, 'newly': 21626, 'eloped': 10420, 'couple': 730

## Train

In [36]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(1600, 36094)


,00,000,0009f,007,03,04,05,05425,10,100,...,zuehlke,zuko,zukovsky,zulu,zurg,zus,zweibel,zwick,zwigoff,zycie
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043066,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Test

In [37]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(400, 36094)


,00,000,0009f,007,03,04,05,05425,10,100,...,zuehlke,zuko,zukovsky,zulu,zurg,zus,zweibel,zwick,zwigoff,zycie
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Logistic Regression

In [38]:
LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

/anaconda3/envs/sandbox/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train Accuracy: 0.98
Test Accuracy: 0.8175


## Multinomial Naive Bayes

In [39]:
MNB = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.96125
Test Accuracy: 0.755


## Random Forest Classifier

In [40]:
RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

/anaconda3/envs/sandbox/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train Accuracy: 0.993125
Test Accuracy: 0.63


# Using NLTK to clean the data

## Importing the data fresh to avoid variable collisions

In [ ]:
reviews = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]

random.shuffle(reviews, )

In [ ]:
documents = []
sentiments = []

for review in reviews:
  
  # Add sentiment to list
  if review[1] == "pos":
    sentiments.append(1)
  else:
    sentiments.append(0)
  
  # Add text to list
  review_text = " ".join(review[0])
  documents.append(review_text)
  
df = pd.DataFrame({"text": documents, "sentiment": sentiments})
df.head()

## Cleaning function to apply to each document

In [ ]:
from nltk.corpus import stopwords
import string

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

df_nltk = pd.DataFrame()
df_nltk['text'] = df.text.apply(clean_doc)
df_nltk['sentiment'] = df.sentiment
df_nltk.head()

## Reformat reviews for sklearn

In [ ]:
documents = []
for review in df_nltk.text:
  review = " ".join(review)
  documents.append(review)
  
sentiment = list(df_nltk.sentiment)
new_df = pd.DataFrame({'text': documents, 'sentiment': sentiment})
new_df.head()

## Train Test Split

In [ ]:
X = new_df.text
y = new_df.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Vectorize the reviews

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)


In [ ]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

In [ ]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

## Logistic Regression

In [ ]:
LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

## Multinomial Naive Bayes

In [ ]:
MNB = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

## Random Forest Classifier

In [ ]:
RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')